# Assignment:3  Selenium Exception Handling 

In [4]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
import requests
from bs4 import BeautifulSoup
import re

1.Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [88]:
from selenium.common.exceptions import InvalidArgumentException

In [116]:
driver=webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")
driver.get('https://www.amazon.in/')
inputU = input('please enter product here--->')

please enter product here--->Headphone 


In [117]:
search_bar = driver.find_element(By.XPATH,'//*[@id="twotabsearchtextbox"]')    # Finding the search bar using it's xpath
search_bar.send_keys(inputU)       # Inputing keyword to search 
search_button = driver.find_element(By.XPATH,'//*[@id="nav-search-submit-button"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [118]:
productName=[]

In [120]:
#scraping the Product_Name 
PName=driver.find_elements(By.XPATH,"//span[@class='a-size-medium a-color-base a-text-normal']")
for i in PName:
    if i.text is None :
        productName.append("--") 
    else:
        productName.append(i.text)
print(len(productName),productName)

24 ['JBL Tune 760NC, Over Ear Active Noise Cancellation Headphones with Mic, up to 50 Hours Playtime, JBL Pure Bass, Google Fast Pair, Dual Pairing, AUX & Voice Assistant Support for Mobile Phones (Black)', 'New JBL Tune 130NC TWS | Active Noise Cancellation Earbuds (Upto 40dB) | JBL APP - Adjust EQ for Extra Bass | Massive 40Hrs Playtime | Legendary JBL Sound | 4Mics for Clear Calls | BT 5.2 (Black)', 'boAt Rockerz 450 Bluetooth On Ear Headphones with Mic, Upto 15 Hours Playback, 40MM Drivers, Padded Ear Cushions, Integrated Controls and Dual Modes(Luscious Black)', 'boAt Rockerz 450 Bluetooth On Ear Headphones with Mic, Upto 15 Hours Playback, 40MM Drivers, Padded Ear Cushions, Integrated Controls and Dual Modes(Aqua Blue)', 'ZEBRONICS Zeb-Thunder Wireless BT Headphone Comes with 40mm Drivers, AUX Connectivity, Built in FM, Call Function, 9Hrs* Playback time and Supports Micro SD Card(Sea Green)', 'Newly Launched pTron Bassbuds Eon TWS Earbuds with ENC, 13mm Driver, Immersive Sound, 

2.In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [121]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-button"]')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element(By.XPATH,'//span[@class="s-pagination-item s-pagination-selected"]').text== 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next pag

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [122]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [125]:
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element(By.XPATH,'//td[@class="a-size-base prodDetAttrValue"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element(By.XPATH,'//span[@id="productTitle"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element(By.XPATH,'//i[@class="a-icon a-icon-star a-star-4-5"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element(By.XPATH,'//span[@id="a-price-whole"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element(By.XPATH,'//a[@class="a-size-small a-link-normal a-text-normal"]')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element(By.XPATH,'//a[@class="a-size-small a-link-normal a-text-normal"]')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element(By.XPATH,'//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element(By.XPATH,'//h1[@class="a-size-medium a-spacing-small"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MzAxMDY1NzEwMjcxMjcyOjE2NjIwMjg2NDU6c3BfYXRmOjIwMDI1ODAxNTEyNzk4OjowOjo&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&url=%2FJBL-Cancellation-Headphones-Playtime-Assistant%2Fdp%2FB096FYLJ6M%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dheadphones%26qid%3D1662028645%26sr%3D8-1-spons%26psc%3D1
Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MzAxMDY1NzEwMjcxMjcyOjE2NjIwMjg2NDU6c3BfYXRmOjIwMDQ2NTg1MzY2NTk4OjowOjo&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&url=%2FJBL-130NC-Active-Cancellation-Earbuds%2Fdp%2FB09HGW9V7D%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dheadphones%26qid%3D1662028645%26smid%3DA14CZOWI0VEHLG%26sr%3D8-2-spons%26psc%3D1
Scraping URL =  https://www.amazon.in/Rockerz-450-Wireless-Bluetooth-Headphone/dp/B07PR1CL3S/ref=sr_1_3?keywords=headphones&qid=1662028645&sr=8-3
Scraping URL =  https://www.amazon.in/Boat-Rockerz-450-Lightweight-Compatibility/dp/B08667772G/ref=sr_1_omk_4?keywords=headphones&qid=1662028645&sr=8-4


In [126]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,JBL,-,,-,"5,999",Cash on Delivery,Cash on Delivery,In stock.,Technical Details,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,JBL,-,,-,,Cash on Delivery,Cash on Delivery,,Technical Details,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,BoAt,-,-,-,"1,499",Cash on Delivery,Cash on Delivery,In stock.,Technical Details,https://www.amazon.in/Rockerz-450-Wireless-Blu...
3,BoAt,-,,-,"1,499",Cash on Delivery,Cash on Delivery,In stock.,Technical Details,https://www.amazon.in/Boat-Rockerz-450-Lightwe...


3.Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [41]:
driver=webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")
driver.get('https://images.google.com/')


In [43]:
search_bar = driver.find_element(By.XPATH,'//*[@class="gLFyf gsfi"]') # Finding the search bar using it's xpath
search_bar.send_keys("fruits")
delay=10  # Inputing "fruits" keyword to search rock images
search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]') 
delay=10   # Finding the xpath of search button
search_button.click()
time.sleep(3)# Clicking the search button

In [44]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page...


In [48]:
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

In [49]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

39

In [50]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response=requests.get(img_urls[i])
    file=open(r"C:\Users\pranay\Desktop" + str(i)+".jpg","wb")
    file.write(response.content)

4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [58]:
driver=webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")
driver.get("https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off")
time.sleep(3)

In [59]:
Brand_Name=[]
Colour=[]
Storage_RAM_ROM=[]
P_F_Camera=[]
Display_size_Resolution=[]
ProcessorAndCores=[]
Battery=[]
Price=[]
Product_URL=[]

In [61]:
#scraping the Brand_Name 
BName=driver.find_elements(By.XPATH,"//div[@class='_4rR01T']")
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)
print(len(Brand_Name),Brand_Name)

24 ['realme C21Y (Cross Black, 64 GB)', 'realme C21Y (Cross Blue, 32 GB)', 'realme C21Y (Cross Black, 32 GB)', 'APPLE iPhone 11 (Black, 128 GB)', 'realme C11 2021 (Cool Blue, 32 GB)', 'realme C11 2021 (Cool Grey, 32 GB)', 'realme 8i (Space Purple, 64 GB)', 'APPLE iPhone 11 (White, 128 GB)', 'realme C21Y (Cross Blue, 64 GB)', 'realme C11 2021 (Cool Blue, 64 GB)', 'Infinix HOT 12 Play (Champagne Gold, 64 GB)', 'realme C11 2021 (Cool Grey, 64 GB)', 'realme 8i (Space Purple, 128 GB)', 'realme 8 (Cyber Silver, 128 GB)', 'realme 8s 5G (Universe Blue, 128 GB)', 'Infinix HOT 12 Play (Horizon Blue, 64 GB)', 'realme 8 (Cyber Black, 128 GB)', 'realme 8s 5G (Universe Purple, 128 GB)', 'realme C31 (Dark Green, 32 GB)', 'realme C31 (Light Silver, 32 GB)', 'Realme C30 - Locked with Airtel Prepaid', 'realme 9 5G (Stargaze White, 64 GB)', 'realme 9 5G (Meteor Black, 64 GB)', 'realme 9 5G (Supersonic Black, 64 GB)']


In [62]:
#scraping the Storage_RAM_ROM 
ram=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[1]")
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text)
print(len(Storage_RAM_ROM),Storage_RAM_ROM)

24 ['4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '3 GB RAM | 32 GB ROM | Expandable Upto 256 GB', '3 GB RAM | 32 GB ROM | Expandable Upto 256 GB', '128 GB ROM', '2 GB RAM | 32 GB ROM | Expandable Upto 256 GB', '2 GB RAM | 32 GB ROM | Expandable Upto 256 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '6 GB RAM | 128 GB ROM | Expandable Upto 256 GB', '6 GB RAM | 128 GB ROM | Expandable Upto 256 GB', '8 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '6 GB RAM | 128 GB ROM | Expandable Upto 256 GB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '2 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 

In [64]:
#scraping the P_F_Camera 
PC=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[3]")
for i in PC:
    if i.text is None :
        P_F_Camera.append("--") 
    else:
        P_F_Camera.append(i.text)
print(len(P_F_Camera),P_F_Camera)

24 ['13MP + 2MP + 2MP | 5MP Front Camera', '13MP + 2MP + 2MP | 5MP Front Camera', '13MP + 2MP + 2MP | 5MP Front Camera', '12MP + 12MP | 12MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '50MP + 2MP + 2MP | 16MP Front Camera', '12MP + 12MP | 12MP Front Camera', '13MP + 2MP + 2MP | 5MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '13MP + Depth Lens | 8MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '50MP + 2MP + 2MP | 16MP Front Camera', '64MP + 8MP + 2MP + 2MP | 16MP Front Camera', '64MP + 2MP + 2MP | 16MP Front Camera', '13MP + Depth Lens | 8MP Front Camera', '64MP + 8MP + 2MP + 2MP | 16MP Front Camera', '64MP + 2MP + 2MP | 16MP Front Camera', '13MP + 2MP + 0.3MP | 5MP Front Camera', '13MP + 2MP + 0.3MP | 5MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '48MP + 2MP + 2MP | 16MP Front Camera', '48MP + 2MP + 2MP | 16MP Front Camera', '48MP + 2MP + 2MP | 16MP Front Camera']


In [65]:
#scraping the Display_size_Resolution 
DS=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[2]")
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
print(len(Display_size_Resolution),Display_size_Resolution)

24 ['16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '15.49 cm (6.1 inch) Liquid Retina HD Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '15.49 cm (6.1 inch) Liquid Retina HD Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '17.32 cm (6.82 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '16.26 cm (6.4 inch) Full HD+ Display', '16.51 cm (6.5 inch) Full HD+ Display', '17.32 cm (6.82 inch) HD+ Display', '16.26 cm (6.4 inch) Full HD+ Display', '16.51 cm (6.5 inch) Full HD+ Display', '16.56 cm (6.52 inch) HD Display', '16.56 cm (6.52 inch) HD Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) Full HD+ Display', '16.51 cm (6.5 inch) Full HD+ Display', '16.51 cm (6.5 inch) Full HD+ Display']


In [66]:
#scraping the ProcessorAndCores 
P=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[5]")
for i in P:
    if i.text is None :
        ProcessorAndCores.append("--") 
    else:
        ProcessorAndCores.append(i.text)
print(len(ProcessorAndCores),ProcessorAndCores)

24 ['Unisoc T610 Processor', 'Unisoc T610 Processor', 'Unisoc T610 Processor', 'Brand Warranty of 1 Year', 'Octa-core Processor', 'Octa-core Processor', 'MediaTek Helio G96 Processor', 'Brand Warranty of 1 Year', 'Unisoc T610 Processor', 'Octa-core Processor', 'Unisoc T610 Processor', 'Octa-core Processor', 'MediaTek Helio G96 Processor', 'MediaTek Helio G95 Processor', 'MediaTek Dimensity 810 5G Processor', 'Unisoc T610 Processor', 'MediaTek Helio G95 Processor', 'MediaTek Dimensity 810 5G Processor', 'Unisoc T612 Processor', 'Unisoc T612 Processor', 'Unisoc T612 Processor', 'Mediatek Dimensity 810 Processor', 'Mediatek Dimensity 810 Processor', 'Mediatek Dimensity 810 Processor']


In [67]:
#scraping the Battery 
B=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[4]")
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)
print(len(Battery),Battery)

24 ['5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', 'A13 Bionic Chip Processor', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', 'A13 Bionic Chip Processor', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Li-ion Polymer Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Li-ion Polymer Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Lithium Ion Battery', '5000 mAh Lithium Polymer Battery', '5000 mAh Lithium Polymer Battery', '5000 mAh Lithium Polymer Battery']


In [68]:
#scraping the Price 
price=driver.find_elements(By.XPATH,"//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
print(len(Price),Price)

24 ['₹10,599', '₹9,499', '₹9,499', '₹46,999', '₹7,499', '₹7,499', '₹14,049', '₹46,999', '₹10,599', '₹8,999', '₹8,999', '₹8,999', '₹16,009', '₹17,049', '₹20,199', '₹8,999', '₹17,049', '₹18,049', '₹9,299', '₹9,299', '₹6,899', '₹16,049', '₹16,049', '₹15,999']


In [69]:
FlipKart=pd.DataFrame([])
FlipKart['Brand_Name']=Brand_Name
FlipKart['Storage_RAM_ROM']=Storage_RAM_ROM
FlipKart['Amount P_F_Camera']=P_F_Camera
FlipKart['Display_size_Resolution']=Display_size_Resolution
FlipKart['ProcessorAndCores']=ProcessorAndCores
FlipKart['Battery']=Battery
FlipKart['Price']=Price

FlipKart

,Brand_Name,Storage_RAM_ROM,Amount P_F_Camera,Display_size_Resolution,ProcessorAndCores,Battery,Price
0,"realme C21Y (Cross Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,13MP + 2MP + 2MP | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Unisoc T610 Processor,5000 mAh Battery,"₹10,599"
1,"realme C21Y (Cross Blue, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,13MP + 2MP + 2MP | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Unisoc T610 Processor,5000 mAh Battery,"₹9,499"
2,"realme C21Y (Cross Black, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,13MP + 2MP + 2MP | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Unisoc T610 Processor,5000 mAh Battery,"₹9,499"
3,"APPLE iPhone 11 (Black, 128 GB)",128 GB ROM,12MP + 12MP | 12MP Front Camera,15.49 cm (6.1 inch) Liquid Retina HD Display,Brand Warranty of 1 Year,A13 Bionic Chip Processor,"₹46,999"
4,"realme C11 2021 (Cool Blue, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Octa-core Processor,5000 mAh Battery,"₹7,499"
5,"realme C11 2021 (Cool Grey, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Octa-core Processor,5000 mAh Battery,"₹7,499"
6,"realme 8i (Space Purple, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,50MP + 2MP + 2MP | 16MP Front Camera,16.76 cm (6.6 inch) Full HD+ Display,MediaTek Helio G96 Processor,5000 mAh Battery,"₹14,049"
7,"APPLE iPhone 11 (White, 128 GB)",128 GB ROM,12MP + 12MP | 12MP Front Camera,15.49 cm (6.1 inch) Liquid Retina HD Display,Brand Warranty of 1 Year,A13 Bionic Chip Processor,"₹46,999"
8,"realme C21Y (Cross Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,13MP + 2MP + 2MP | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Unisoc T610 Processor,5000 mAh Battery,"₹10,599"
9,"realme C11 2021 (Cool Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Octa-core Processor,5000 mAh Battery,"₹8,999"


5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [19]:
driver=webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")
driver.get("https://www.google.co.in/maps")
time.sleep(3)

In [20]:
city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element(By.ID,"searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element(By.ID,"searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : Nagpur
URL Extracted:  https://www.google.co.in/maps/place/Nagpur,+Maharashtra/@21.1610282,78.9324187,11z/data=!3m1!4b1!4m5!3m4!1s0x3bd4c0a5a31faf13:0x19b37d06d0bb3e2b!8m2!3d21.1458004!4d79.0881546
Latitude = 21.1610282, Longitude = 78.9324187


6.Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [140]:
driver=webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")
driver.get("https://trak.in/india-startup-funding-investment-2015/")

In [38]:
Dates=[]
Company=[]
Industry=[]
Investor_Name=[]
Investment_Type=[]
Amount=[]

In [40]:
#scraping the company_name 
companies=driver.find_elements(By.XPATH,"//td[@class='column-3']")
for i in companies:
    if i.text is None :
        Company.append("--") 
    else:
        Company.append(i.text)
print(len(Company),Company)

392 ['BYJU’S', 'Meesho', 'Swiggy', 'Groww', 'Beldara', 'DealShare', 'Uniphore', 'Dunzo', 'BYJU’S', 'SkilloVilla', 'CityMall', 'DotPe', 'Doubtnut', 'Zomato', 'Fingerlix', 'Zolve', 'KreditBee', 'Pepperfry', 'Grofers', 'Nothing', 'SplashLearn', 'Digit Insurance', 'Bombay Shaving Company', 'DeHaat', 'Darwinbox', 'mfine', 'Udayy', 'True Elements', 'Saveo', 'Bira 91', 'Pine Labs', 'Zenoti', 'Tax Buddy India', 'Delhivery', 'PagarBook', 'Codingal', 'CRED', 'Bira 91', 'Chumbak', 'Cityflo', 'Shipsy', 'Unacademy', 'Cashfree', 'Credgenics', 'Country Delight', 'Nykaa', 'Betterplace', 'FlexiLoans', 'GetVantage', 'FreshToHome', 'Origo', 'Origo', 'Treebo Hotels', 'Hubilo', 'CredAble', 'Byju’s', 'mCaffeine', 'Qshala', 'Winzo', 'Hippo Video', 'Melorra', '1mg', 'mfine', 'Apna', 'Railofy', 'Practo', 'Medlife', 'HungerBox', 'Dunzo', 'Terra.do', 'Classplus', 'Niyo', 'ZestMoney', 'FreshToHome', 'Eduvanz', 'Flipkart', 'Vedantu', 'Crio', 'goDutch', 'Mystifly', 'JetSynthesys', 'gigIndia', 'PumPumPum', 'FLYX', '

In [41]:
#scraping the Industry 
Ind=driver.find_elements(By.XPATH,"//td[@class='column-4']")
for i in Ind:
    if i.text is None :
        Industry.append("--") 
    else:
        Industry.append(i.text)
print(len(Industry),Industry)

392 ['Edu-tech', 'E-commerce', 'Online Food Delivery', 'FinTech', 'E-commerce', 'E-commerce', 'Technology', 'E-commerce', 'Edu-tech', 'Edu-tech', 'E-commerce', 'FinTech', 'Edu Tech', 'Hospitality', 'Hospitality', 'FinTech', 'Finance', 'E-commerce', 'E-Commerce', 'Technology', 'EdTech', 'Financial Services', 'Consumer Goods Company', 'AgriTech Startup', 'SaaS', 'Health Tech Startup', 'EdTech', 'Food Startup', 'B2B E-commerce', 'Food and Beverage', 'Financial Services', 'Technology Provider', 'FinTech', 'Supply chain', 'SaaS', 'EduTech', 'Financial Services', 'Food and Beverage', 'Lifestyle', 'Transportation', 'SaaS', 'Edutech', 'FinTech', 'Fintech', 'Dairy Tech', 'E-commerce', 'Digital Solutions', 'FinTech', 'FinTech', 'E-commerce', 'AgriTech', 'Advertising', 'Hospitality', 'Information Technology', 'FinTech', 'EduTech', 'Personal Care', 'EduTech', 'Online Gaming', 'Video Customer Experience(CX) Platform', 'E-commerce', 'E-commerce', 'HealthTech', 'Human Resources', 'Transportation', 'H

In [139]:
#scraping the Dates 
dt=driver.find_elements(By.XPATH,"//td[@class='column-2']")
for i in dt:
    if i.text is None :
        Dates.append("--") 
    else:
        Dates.append(i.text)
print(len(Dates),Dates)

392 ['01/04/2021', '05/04/2021', '14/04/2021', '07/04/2021', '14/04/2021', '04/03/2021', '31/03/2021', '30/03/2021', '30/03/2021', '23/03/2021', '25/03/2021', '26/03/2021', '11/02/2021', '22/02/2021', '19/02/2021', '17/02/2021', '15/02/2021', '12/02/2021', '12/02/2021', '09/02/2021', '09/02/2021', '15/01/2021', '28/01/2021', '19/01/2021', '19/01/2021', '18/01/2021', '18/01/2021', '11/01/2021', '13/01/2021', '15/12/2020', '21/12/2020', '15/12/2020', '22/12/2020', '15/12/2020', '18/12/2020', '30/11/2020', '30/11/2020', '29/11/2020', '27/11/2020', '25/11/2020', '25/11/2020', '25/11/2020', '24/11/2020', '30/11/2020', '23/11/2020', '28/10/2020', '28/10/2020', '28/10/2020', '27/10/2020', '27/10/2020', '27/10/2020', '27/10/2020', '15/10/2020', '26/10/2020', '28/10/2020', '08/09/2020', '12/09/2020', '09/09/2020', '02/09/2020', '09/09/2020', '07/09/2020', '07/09/2020', '31/08/2020', '31/08/2020', '03/09/2020', '15/08/2020', '13/08/2020', '13/08/2020', '04/08/2020', '11/08/2020', '12/08/2020', '

In [43]:
#scraping the Investor_Name 
IN=driver.find_elements(By.XPATH,"//td[@class='column-7']")
for i in IN:
    if i.text is None :
        Investor_Name.append("--") 
    else:
        Investor_Name.append(i.text)
print(len(Investor_Name),Investor_Name)

392 ['Innoven Capital', 'SoftBank Vision Fund 2', 'Amansa Holdings, Carmignac, Falcon Edge Capital, Goldman Sachs, Think Investments', 'MC Global Edtech, B Capital, Baron, others', 'Hindustan Media Ventures', 'Innoven Capital', 'Sorenson Capital Partners', 'Krishtal Advisors Pte Ltd', 'MC Global Edtech, B Capital, Baron, others', 'Titan Capital, others', 'Accel Partners', 'PayU', 'SIG Global, Sequoia Capital, WaterBridge Ventures and ON Mauritius', 'Tiger Global, Kora', 'Rhodium Trust, Accel Partners and Swiggy', 'Accel Partners and Lightspeed Venture Partners', 'Azim Premji’s PremjiInvest and South Korea’s Mirae Asset Venture', 'InnoVen Capital', 'SoftBank Vision Fund (SVF)', 'GV', 'Owl Ventures', 'A91 Partners, Faering Capital, TVS Capital Funds', 'Reckitt Benckiser', 'Prosus Ventures', 'Salesforce Ventures', 'Heritas Capital Management', 'Sequoia Capital', 'SIDBI Venture Capital', 'Matrix Partners India, RTP Global, others', 'Nishant Mittal, Pankaj Chaddah and Shikha Sunil Chandak',

In [44]:
#scraping the Investment_Type 
IT=driver.find_elements(By.XPATH,"//td[@class='column-8']")
for i in IT:
    if i.text is None :
        Investment_Type.append("--") 
    else:
        Investment_Type.append(i.text)
print(len(Investment_Type),Investment_Type)

392 ['Series F', 'Series E', 'Series J', 'Series D', 'Venture', 'Debt Financing', 'Series D', 'Series E', 'Series F', 'Seed', 'Series A', 'Series A', 'Series B', 'Venture', 'Series C', 'Seed', 'Series C', 'Debt Financing', 'Unspecified', 'Series A', 'Series C', 'Venture', 'Venture', 'Series C', 'Seed', 'Venture Round', 'Seed Funding', 'Series', 'Seed', 'Funding', 'Secondary Market', 'Series D', 'Seed', 'Secondary Market', 'Series A', 'Seed', 'Series C', 'Bridge Funding', 'Pre-series E', 'Series A', 'Series A', 'Venture', 'Series B', 'Pre-Series A', 'Series C', 'Personal Investment', 'Series B', 'Equity and Debt', 'Seed', 'Series C', 'Debt', 'Series A', '', 'Seed', 'Follow-On', 'Private Equity', 'Series B', 'Angel', 'Series B', 'Series A', 'Debt Financing', 'In Progress', 'Series B', 'Series A', 'Seed', 'Series F', '', 'Series D1', 'In Progress', 'Seed', 'In Progress', '', '', 'Venture', 'Series A', 'M&A', 'Series D', 'pre-Series A', 'Seed', 'pre-Series B', 'Venture-Series Unknown', 'pr

In [45]:
#scraping the Amount 
Price=driver.find_elements(By.XPATH,"//td[@class='column-9']")
for i in Price:
    if i.text is None :
        Amount.append("--") 
    else:
        Amount.append(i.text)
print(len(Amount),Amount)

392 ['460,000,000', '300,000,000', '343,000,000', '83,000,000', '7,400,000', '250,000,000', '140,000,000', '8,000,000', '460,000,000', '300,000,000', '11,000,000', '27,500,000', '2,500,000', '250,000,000', '2,747,045.20', '1,50,00,000', '75,000,000', '4,773,958', '55,000,000', '15,000,000', '18,000,000', '1,80,00,000', '6,172,258.50', '30,000,000', '15,000,000', '16,000,000', '15,000,000', '100,000,000', '4,000,000', '1,273,634.07', 'Undisclosed', '160,000,000', '1,000,000', '25,000,000', '15,000,000', '560,000', '80,000,000', '3,000,000', '1,001,047.92', '7,700,000', '6,003,806', '75,000,000-100,000,000', '35,300,000', '3,500,000', '25,000,000', 'NA', '10,000,000', '20,160,000', '5,000,000', '121,000,000', '4,700,000', '15,000,000', '6,000,000', '4,500,000', '4,500,000', '500,000,000', '3,000,000', '370,000', '15,500,000', '4,500,000', 'upto 8,900,000', '100,000,000', '5,400,000', '8,000,000', '950,000', '32,000,000', '23,000,000', '1,560,000', '30,000,000', '1,400,000', 'upto 15,000,

In [46]:
Funding=pd.DataFrame([])
Funding['Company']=Company
Funding['Industry']=Industry
Funding['Investor_Name']=Investor_Name
Funding['Amount Invested']=Amount
Funding['Specification']=Investment_Type
Funding['Dates']=Dates
Funding

,Company,Industry,Investor_Name,Amount Invested,Specification,Dates
0,BYJU’S,Edu-tech,Innoven Capital,"460,000,000",Series F,01/04/2021
1,Meesho,E-commerce,SoftBank Vision Fund 2,"300,000,000",Series E,05/04/2021
2,Swiggy,Online Food Delivery,"Amansa Holdings, Carmignac, Falcon Edge Capita...","343,000,000",Series J,14/04/2021
3,Groww,FinTech,"MC Global Edtech, B Capital, Baron, others","83,000,000",Series D,07/04/2021
4,Beldara,E-commerce,Hindustan Media Ventures,"7,400,000",Venture,14/04/2021
...,...,...,...,...,...,...
387,zippserv,Consumer Internet,Info Edge (India) Ltd,"440,000",Seed/ Angel Funding,09/07/2018
388,Groww,Consumer Internet,"Insignia Ventures Partners, Lightbridge Partne...","1,100,000",Seed/ Angel Funding,09/07/2018
389,Avenue Growth,Consumer Internet,"Avtar Monga, chief operating office at IDFC Ba...","437,000",Seed/ Angel Funding,10/07/2018
390,iNICU,Healthcare,Venture Catalysts,N/A,Seed/ Angel Funding,10/07/2018



7.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [381]:
driver=webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")

In [382]:
url="https://www.digit.in/top-products/best-gaming-laptops-40.html"

In [383]:
driver.get(url)

In [384]:
Brands=[]
Products_Description=[]
Specification=[]
Price=[]

In [385]:
br=driver.find_elements(By.XPATH,"//div[@class='TopNumbeHeading active sticky-footer']")
len(br)

10

In [386]:
for i in br:
   
    Brands.append(str(i.text).replace("\n",""))
Brands

['1.MSI TITAN GT77-12UHS',
 '2.ALIENWARE X17 R2',
 '3.ACER PREDATOR TRITON 500 SE PT516-52S',
 '4.OMEN BY HP (16-B1371TX)',
 '5.ACER PREDATOR HELIOS 300 AN515-45 (NH.QBRSI.0',
 '6.MSI DELTA 15 (A5EFK-083IN)',
 '7.OMEN BY HP (16-C0141AX)',
 '8.LENOVO LEGION 5I PRO (82RF00MGIN)',
 '9.ALIENWARE M15 R5 RYZEN EDITION ICC-C780001WIN',
 '10.LENOVO SLIM 7 GEN 6 (82K8002JIN)']

In [387]:
sp=driver.find_elements(By.XPATH,"//div[@class='Specs-Wrap']")
len(sp)

10

In [388]:
for i in sp:
   
    Specification.append(str(i.text).replace("\n",""))
Specification

['WINDOWS 11 HOME OS17.3" (3840 X 2160) DISPLAY12TH GEN INTEL CORE I9-12900HX | NA PROCESSOR2 TB SSD/64 GB DDR5 MEMORY',
 'WINDOWS 11 HOME OS17.3" (1920X1080) DISPLAY12TH GEN INTEL CORE I9-12900HK | 5.00 GHZ PROCESSOR1 TB SSD/32 GB DDR5 MEMORY',
 'WINDOWS 11 HOME OS16" (2560 X 1600) DISPLAY12TH GEN INTEL CORE I7-12700H | 3.50 GHZ PROCESSOR2 TB SSD/32 GB DDR5 MEMORY',
 'WINDOWS 11 HOME OS16.1" (2560 X 1440) DISPLAY12TH GEN INTEL CORE I7-12700H | 4.7 GHZ PROCESSOR1 TB SSD/8 GB DDR5 MEMORY',
 'WINDOWS 11 HOME OS15.6" (2560 X 1440) DISPLAYAMD RYZEN 9-5900HX | NA PROCESSOR512 GB SSD/16 GB DDR4 MEMORY',
 'WINDOWS 11 HOME OS15.6" (1920 X 1080) DISPLAYAMD 5TH GEN RYZEN 9-5900HX | 3.3GHZ PROCESSOR1 TB SSD/16 GBGB DDR4 MEMORY',
 'WINDOWS 11 HOME OS16.1" (2560 X 1440) DISPLAYAMD RYZEN™ 9 5900HX | 3.3 GHZ PROCESSOR1 TB NVME/16 GB DDR4 MEMORY',
 'WINDOWS 11 HOME OS16" (2560 X 1600) DISPLAY12TH GEN INTEL CORE I7-12700H | 3.50 GHZ PROCESSOR1 TB SSD/16 GB DDR5 MEMORY',
 'WINDOWS 11 HOME OS15.6" (1920 

In [389]:
des=driver.find_elements(By.XPATH,"//div[@class='Section-center']")
len(des)

10

In [390]:
for i in des:
   
    Products_Description.append(str(i.text).replace("\n",""))
Products_Description

['The very best MSI gaming laptop featuring an Intel processor is the MSI TITAN GT77 - 12UHS. This is the top of the line model from MSI’s recent refresh of their entire gaming lineup and is powered by the Intel Core i9-12900HX which is a 16 Core (8P+8E) processor that has 24 threads and a maximum Turbo power of 157 Watts. The graphics card in this laptop is the NVIDIA RTX 3080 Ti and it too sports a high TGP of 175 Watts. This is one of the most powerful CPU+GPU combinations with plenty of thermal headroom to allow it to operate at higher clock speeds for longer time periods. As for the memory, the MSI GT77-12UHS can have up to 128 GB of DDR5 memory but ships with 64 GB DDR5 clocked at 4800 MT/s. Being a 12th Gen machine also means that the motherboard has PCIe Gen 5.0 lanes for storage devices such as M.2 SSDs. The GT77-12UHS has four M.2 slots of which one slot is PCIe Gen 5 whereas the remaining three are PCIe Gen 4 slots. Then there’s also the display, the GT77-12UHS features a ma

In [392]:
pri=driver.find_elements(By.XPATH,"//td[@class='smprice']")
len(pri)

10

In [393]:
for i in pri:
   
    Price.append(str(i.text).replace("\n",""))
Price

['N/A',
 '₹ 389,990',
 '₹ 300,000',
 '₹ 194,000',
 '₹ 172,999',
 '₹ 188,990',
 '₹ 129,899',
 '₹ 230,890',
 '₹ 144,990',
 '₹ 131,990']

In [394]:
digit_lap=pd.DataFrame([])
digit_lap['Brands']=Brands[0:10]
digit_lap['Price']=Price[0:10]
digit_lap['Specification']=Specification[0:10]
digit_lap['Description']=Products_Description[0:10]
digit_lap

,Brands,Price,Specification,Description
0,1.MSI TITAN GT77-12UHS,N/A,"WINDOWS 11 HOME OS17.3"" (3840 X 2160) DISPLAY1...",The very best MSI gaming laptop featuring an I...
1,2.ALIENWARE X17 R2,"₹ 389,990","WINDOWS 11 HOME OS17.3"" (1920X1080) DISPLAY12T...","Alienware, the premium gaming brand from Dell ..."
2,3.ACER PREDATOR TRITON 500 SE PT516-52S,"₹ 300,000","WINDOWS 11 HOME OS16"" (2560 X 1600) DISPLAY12T...",The Predator lineup from Acer has shown great ...
3,4.OMEN BY HP (16-B1371TX),"₹ 194,000","WINDOWS 11 HOME OS16.1"" (2560 X 1440) DISPLAY1...",Omen by HP is a premium gaming lineup which is...
4,5.ACER PREDATOR HELIOS 300 AN515-45 (NH.QBRSI.0,"₹ 172,999","WINDOWS 11 HOME OS15.6"" (2560 X 1440) DISPLAYA...",Acer Predator Helios 300 AN515-45 is the best ...
5,6.MSI DELTA 15 (A5EFK-083IN),"₹ 188,990","WINDOWS 11 HOME OS15.6"" (1920 X 1080) DISPLAYA...",The best MSI gaming laptop with an AMD process...
6,7.OMEN BY HP (16-C0141AX),"₹ 129,899","WINDOWS 11 HOME OS16.1"" (2560 X 1440) DISPLAYA...",Another worthy gaming laptop from the Omen lin...
7,8.LENOVO LEGION 5I PRO (82RF00MGIN),"₹ 230,890","WINDOWS 11 HOME OS16"" (2560 X 1600) DISPLAY12T...",Lenovo has gone a little mainstream with their...
8,9.ALIENWARE M15 R5 RYZEN EDITION ICC-C780001WIN,"₹ 144,990","WINDOWS 11 HOME OS15.6"" (1920 X 1080) DISPLAYA...",Dell not only has gaming laptops under the Del...
9,10.LENOVO SLIM 7 GEN 6 (82K8002JIN),"₹ 131,990",WINDOWS 11 HOME OS15.6 MP | NA DISPLAYLENOVO R...,Lenovo has been pushing for thinner gaming lap...


8.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [48]:
#gedit web-s.py

from selenium import webdriver

import pandas as pd
driver = webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")
driver.get("https://www.forbes.com/billionaires/")
Name=[] #List to store name of the product
Rank=[] #List to store price of the product
Networth=[] #List to store rating of the product
Country=[] #List to store name of the product
Industry=[] #List to store price of the product


In [50]:
rank=driver.find_elements(By.XPATH,"//div[@class='rank']")
len(rank)

200

In [51]:
for i in rank:
   
    Rank.append(str(i.text).replace("\n",""))
Rank

['1.',
 '2.',
 '3.',
 '4.',
 '5.',
 '6.',
 '7.',
 '8.',
 '9.',
 '10.',
 '11.',
 '12.',
 '13.',
 '14.',
 '15.',
 '16.',
 '17.',
 '18.',
 '19.',
 '19.',
 '21.',
 '21.',
 '23.',
 '24.',
 '25.',
 '26.',
 '27.',
 '28.',
 '29.',
 '30.',
 '31.',
 '32.',
 '33.',
 '34.',
 '35.',
 '36.',
 '37.',
 '37.',
 '39.',
 '40.',
 '41.',
 '41.',
 '43.',
 '43.',
 '45.',
 '46.',
 '47.',
 '48.',
 '49.',
 '50.',
 '51.',
 '52.',
 '53.',
 '54.',
 '55.',
 '56.',
 '56.',
 '56.',
 '59.',
 '60.',
 '61.',
 '62.',
 '63.',
 '64.',
 '65.',
 '66.',
 '67.',
 '67.',
 '69.',
 '70.',
 '71.',
 '71.',
 '73.',
 '74.',
 '75.',
 '76.',
 '77.',
 '78.',
 '79.',
 '80.',
 '81.',
 '82.',
 '83.',
 '84.',
 '85.',
 '85.',
 '87.',
 '88.',
 '89.',
 '90.',
 '91.',
 '91.',
 '91.',
 '94.',
 '94.',
 '96.',
 '97.',
 '98.',
 '99.',
 '100.',
 '100.',
 '102.',
 '103.',
 '104.',
 '104.',
 '106.',
 '106.',
 '106.',
 '106.',
 '110.',
 '111.',
 '112.',
 '113.',
 '114.',
 '115.',
 '116.',
 '117.',
 '117.',
 '119.',
 '119.',
 '121.',
 '121.',
 '123.',
 

In [52]:
name=driver.find_elements(By.XPATH,"//div[@class='personName']")
len(name)

200

In [53]:
name=driver.find_elements(By.XPATH,"//div[@class='personName']")
len(name)

200

In [54]:
for i in name:
   
    Name.append(str(i.text).replace("\n",""))
Name

['Elon Musk',
 'Jeff Bezos',
 'Bernard Arnault & family',
 'Bill Gates',
 'Warren Buffett',
 'Larry Page',
 'Sergey Brin',
 'Larry Ellison',
 'Steve Ballmer',
 'Mukesh Ambani',
 'Gautam Adani & family',
 'Michael Bloomberg',
 'Carlos Slim Helu & family',
 'Francoise Bettencourt Meyers & family',
 'Mark Zuckerberg',
 'Jim Walton',
 'Zhong Shanshan',
 'Alice Walton',
 'Rob Walton',
 'Changpeng Zhao',
 'Charles Koch',
 'Julia Koch & family',
 'Amancio Ortega',
 'Michael Dell',
 'Zhang Yiming',
 'David Thomson & family',
 'Phil Knight & family',
 'Dieter Schwarz',
 'Robin Zeng',
 'MacKenzie Scott',
 'Rodolphe Saadé & family',
 'François Pinault & family',
 'Klaus-Michael Kuehne',
 'Ma Huateng',
 'Beate Heister & Karl Albrecht Jr. & family',
 'Giovanni Ferrero',
 'Li Ka-shing',
 'Stephen Schwarzman',
 'Lee Shau Kee',
 'Len Blavatnik',
 'Jacqueline Mars',
 'John Mars',
 'Alain Wertheimer',
 'Gerard Wertheimer',
 'German Larrea Mota Velasco & family',
 'Gina Rinehart',
 'Shiv Nadar',
 'Jim Si

In [56]:
networth=driver.find_elements(By.XPATH,"//div[@class='netWorth']")
len(networth)

200

In [57]:
for i in networth:
   
    Networth.append(str(i.text).replace("\n",""))
Networth

['$219 B',
 '$171 B',
 '$158 B',
 '$129 B',
 '$118 B',
 '$111 B',
 '$107 B',
 '$106 B',
 '$91.4 B',
 '$90.7 B',
 '$90 B',
 '$82 B',
 '$81.2 B',
 '$74.8 B',
 '$67.3 B',
 '$66.2 B',
 '$65.7 B',
 '$65.3 B',
 '$65 B',
 '$65 B',
 '$60 B',
 '$60 B',
 '$59.6 B',
 '$55.1 B',
 '$50 B',
 '$49.2 B',
 '$47.3 B',
 '$47.1 B',
 '$44.8 B',
 '$43.6 B',
 '$41.4 B',
 '$40.4 B',
 '$37.3 B',
 '$37.2 B',
 '$36.8 B',
 '$36.2 B',
 '$34.8 B',
 '$34.8 B',
 '$32.6 B',
 '$32.5 B',
 '$31.7 B',
 '$31.7 B',
 '$31.2 B',
 '$31.2 B',
 '$30.8 B',
 '$30.2 B',
 '$28.7 B',
 '$28.6 B',
 '$28.3 B',
 '$27.5 B',
 '$27.4 B',
 '$27.3 B',
 '$27.2 B',
 '$26.1 B',
 '$25.2 B',
 '$24.3 B',
 '$24.3 B',
 '$24.3 B',
 '$24.1 B',
 '$24 B',
 '$23.9 B',
 '$23.7 B',
 '$23.5 B',
 '$23.2 B',
 '$23.1 B',
 '$23 B',
 '$22.8 B',
 '$22.8 B',
 '$22.3 B',
 '$22.1 B',
 '$22 B',
 '$22 B',
 '$21.8 B',
 '$21.3 B',
 '$21.2 B',
 '$20.8 B',
 '$20.7 B',
 '$20.6 B',
 '$20.3 B',
 '$20.1 B',
 '$20 B',
 '$19.6 B',
 '$19.5 B',
 '$19 B',
 '$18.7 B',
 '$18.7 B',
 '

In [58]:
country=driver.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
len(country)

200

In [59]:
for i in country:
   
    Country.append(str(i.text).replace("\n",""))
Country

['United States',
 'United States',
 'France',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'India',
 'India',
 'United States',
 'Mexico',
 'France',
 'United States',
 'United States',
 'China',
 'United States',
 'United States',
 'Canada',
 'United States',
 'United States',
 'Spain',
 'United States',
 'China',
 'Canada',
 'United States',
 'Germany',
 'Hong Kong',
 'United States',
 'France',
 'France',
 'Germany',
 'China',
 'Germany',
 'Italy',
 'Hong Kong',
 'United States',
 'Hong Kong',
 'United States',
 'United States',
 'United States',
 'France',
 'France',
 'Mexico',
 'Australia',
 'India',
 'United States',
 'China',
 'United States',
 'Austria',
 'Italy',
 'United States',
 'Japan',
 'China',
 'Germany',
 'India',
 'China',
 'China',
 'United States',
 'Japan',
 'China',
 'France',
 'Indonesia',
 'United States',
 'Switzerland',
 'Chile',
 'China',
 'Indonesia',
 'United States',
 'United States',
 'Unite

In [60]:
industry=driver.find_elements(By.XPATH,"//div[@class='source']")
len(industry)

200

In [61]:
for i in industry:
   
    Industry.append(str(i.text).replace("\n",""))
Industry

['Tesla, SpaceX',
 'Amazon',
 'LVMH',
 'Microsoft',
 'Berkshire Hathaway',
 'Google',
 'Google',
 'software',
 'Microsoft',
 'diversified',
 'infrastructure, commodities',
 'Bloomberg LP',
 'telecom',
 "L'Oréal",
 'Facebook',
 'Walmart',
 'beverages, pharmaceuticals',
 'Walmart',
 'Walmart',
 'cryptocurrency exchange',
 'Koch Industries',
 'Koch Industries',
 'Zara',
 'Dell computers',
 'TikTok',
 'media',
 'Nike',
 'retail',
 'batteries',
 'Amazon',
 'shipping',
 'luxury goods',
 'shipping',
 'internet media',
 'supermarkets',
 'Nutella, chocolates',
 'diversified',
 'investments',
 'real estate',
 'music, chemicals',
 'candy, pet food',
 'candy, pet food',
 'Chanel',
 'Chanel',
 'mining',
 'mining',
 'software services',
 'hedge funds',
 'home appliances',
 'casinos',
 'Red Bull',
 'eyeglasses',
 'hedge funds',
 'fashion retail',
 'online games',
 'BMW, pharmaceuticals',
 'vaccines',
 'package delivery',
 'pig breeding',
 'cryptocurrency exchange',
 'sensors',
 'automobiles',
 'chees

In [62]:
World_Billionaire =pd.DataFrame([])
World_Billionaire['Rank']=Rank[0:10]
World_Billionaire['Name']=Name[0:10]
World_Billionaire['Networth']=Networth[0:10]
World_Billionaire['Country']=Country[0:10]
World_Billionaire['Industry']=Industry[0:10]
World_Billionaire

,Rank,Name,Networth,Country,Industry
0,1.,Elon Musk,$219 B,United States,"Tesla, SpaceX"
1,2.,Jeff Bezos,$171 B,United States,Amazon
2,3.,Bernard Arnault & family,$158 B,France,LVMH
3,4.,Bill Gates,$129 B,United States,Microsoft
4,5.,Warren Buffett,$118 B,United States,Berkshire Hathaway
5,6.,Larry Page,$111 B,United States,Google
6,7.,Sergey Brin,$107 B,United States,Google
7,8.,Larry Ellison,$106 B,United States,software
8,9.,Steve Ballmer,$91.4 B,United States,Microsoft
9,10.,Mukesh Ambani,$90.7 B,India,diversified


9.Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('useAutomationExtension', False)
browser = webdriver.Chrome('./chromedriver', options=chrome_options)

browser.get(("https://www.youtube.com/"))



In [12]:
from selenium import webdriver
import time
import os
import csv
import pandas as pd
from math import ceil

In [13]:
# Creates a new .csv file that the data will be written to

csv_file = open('C:\\Users\\pranay\\Desktop\\selenium.csv', 'w', encoding="UTF-8", newline="")
writer = csv.writer(csv_file)

In [14]:
# write header names
writer.writerow(
    ['url', 'link_title', 'channel', 'no_of_views', 'time_uploaded', 'comment', 'author', 'comment_posted', 
     'no_of_replies','upvotes','downvotes'])

112

In [27]:
driver=webdriver.Chrome(r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")

In [34]:
# open chrome 
youtube_pages = "https://www.youtube.com/"
locationOfWebdriver = r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe"
driver = webdriver.Chrome(locationOfWebdriver)
driver.get(youtube_pages)
time.sleep(30)
try:
    print("=" * 40)  # Shows in terminal when youtube summary page with search keyword is being scraped
    print("Scraping " + youtube_pages)
    search = driver.find_element(By.ID,'center')
    search.send_keys("Kishore Kumar")    
    driver.find_element(By.ID,'search-icon-legacy').click()
    time.sleep(20)    
    vtitle = driver.find_elements(By.ID,'video-title')
    subscription = driver.find_elements(By.ID,'byline')
    views = driver.find_elements(By.XPATH,'//div[@id="metadata-line"]/span[1]')
    posted = driver.find_elements(By.XPATH,'//div[@id="metadata-line"]/span[2]')
     
    tcount = 0
    href = []
    title = []
    channel = []
    numview = []
    postdate = []
        
    while tcount < 10:
        href.append(vtitle[tcount].get_attribute('href'))
        channel.append(subscription[tcount].get_attribute('title'))
        title.append(vtitle[tcount].text)
        numview.append(views[tcount].text)
        postdate.append(posted[tcount].text)  
        tcount = tcount +1 
    tcount = 0    
    while tcount < 10:  
        youtube_dict ={}
        # extract comment section of top ten links
        url = href[tcount]
        print (url)
        driver.get(url)
        time.sleep(5)
        
        try:
            print("+" * 40)  # Shows in terminal when details of a new video is being scraped
            print("Scraping child links ")
            #scroll down to load comments
            driver.execute_script('window.scrollBy(0,1000);')
            time.sleep(15)
            #sort by top comments
            sort= driver.find_element(By.XPATH,"""//*[@id="icon-label"]""")
            sort.click()
            time.sleep(10)
            topcomments =driver.find_element(By.XPATH,"""//*[@id="menu"]/a[1]/paper-item/paper-item-body/div[1]""")
            topcomments.click()
            time.sleep(10)
            # Loads 20 comments , scroll two times to load next set of 40 comments. 
            for i in range(0,2):
                driver.execute_script("window.scrollTo(0,Math.max(document.documentElement.scrollHeight,document.body.scrollHeight,document.documentElement.clientHeight))")
                time.sleep(10)
            
            #count total number of comments and set index to number of comments if less than 50 otherwise set as 50. 
            totalcomments= len(driver.find_elements(By.XPATH,"""//*[@id="content-text"]"""))
         
            if totalcomments < 50:
                index= totalcomments
            else:
                index= 50 
                
            ccount = 0
            while ccount < index: 
                try:
                    comment = driver.find_elements(By.XPATH,'//*[@id="content-text"]')[ccount].text
                except:
                    comment = ""
                try:
                    authors = driver.find_elements(By.XPATH,'//a[@id="author-text"]/span')[ccount].text
                except:
                    authors = ""
                try:
                    comment_posted = driver.find_elements(By.XPATH,'//*[@id="published-time-text"]/a')[ccount].text
                except:
                    comment_posted = ""
                try:
                    replies = driver.find_elements(By.XPATH,'//*[@id="more-text"]')[ccount].text                    
                    if replies =="View reply":
                        replies= 1
                    else:
                        replies =replies.replace("View ","")
                        replies =replies.replace(" replies","")
                except:
                    replies = ""
                try:
                    upvotes = driver.find_elements(By.XPATH,'//*[@id="vote-count-middle"]')[ccount].text
                except:
                    upvotes = ""
                        
                youtube_dict['url'] = href[tcount]
                youtube_dict['link_title'] = title[tcount]
                youtube_dict['channel'] = channel[tcount]
                youtube_dict['no_of_views'] = numview[tcount]
                youtube_dict['time_uploaded'] =  postdate[tcount]
                youtube_dict['comment'] = comment
                youtube_dict['author'] = authors
                youtube_dict['comment_posted'] = comment_posted
                youtube_dict['no_of_replies'] = replies
                youtube_dict['upvotes'] = upvotes
                
                writer.writerow(youtube_dict.values())
                ccount = ccount +1
                
        except Exception as e:
            print(e)
            driver.close()
        tcount = tcount +1 
    print("Scrapping process Completed")   
    csv_file.close()    
except Exception as e:
    print(e)
    driver.close()

Scraping https://www.youtube.com/
Message: element not interactable
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x005A78B3+2193587]
	Ordinal0 [0x00540681+1771137]
	Ordinal0 [0x00454070+802928]
	Ordinal0 [0x0047DB83+973699]
	Ordinal0 [0x0047D25C+971356]
	Ordinal0 [0x0049D7AC+1103788]
	Ordinal0 [0x00478C04+953348]
	Ordinal0 [0x0049D9C4+1104324]
	Ordinal0 [0x004ADAE2+1170146]
	Ordinal0 [0x0049D5C6+1103302]
	Ordinal0 [0x004777E0+948192]
	Ordinal0 [0x004786E6+952038]
	GetHandleVerifier [0x00850CB2+2738370]
	GetHandleVerifier [0x008421B8+2678216]
	GetHandleVerifier [0x006317AA+512954]
	GetHandleVerifier [0x00630856+509030]
	Ordinal0 [0x0054743B+1799227]
	Ordinal0 [0x0054BB68+1817448]
	Ordinal0 [0x0054BC55+1817685]
	Ordinal0 [0x00555230+1856048]
	BaseThreadInitThunk [0x76F06739+25]
	RtlGetFullPathName_UEx [0x779B90AF+1215]
	RtlGetFullPathName_UEx [0x779B907D+1165]



In [32]:
driver.find_element(By.ID,'search-icon-legacy').click()
    time.sleep(20)    
    vtitle = driver.find_elements(By.ID,'video-title')
    subscription = driver.find_elements(By.ID,'byline')
    views = driver.find_elements(By.XPATH,'//div[@id="metadata-line"]/span[1]')
    posted = driver.find_elements(By.XPATH,'//div[@id="metadata-line"]/span[2]')
    
    tcount = 0
    href = []
    title = []
    channel = []
    numview = []
    postdate = []
        
    while tcount < 10:
        href.append(vtitle[tcount].get_attribute('href'))
        channel.append(subscription[tcount].get_attribute('title'))
        title.append(vtitle[tcount].text)
        numview.append(views[tcount].text)
        postdate.append(posted[tcount].text)  
        tcount = tcount +1
    
    # launch top ten extracted links and extract comment details
    tcount = 0    
    while tcount < 10:  
        youtube_dict ={}
        # extract comment section of top ten links
        url = href[tcount]
        print (url)
        driver.get(url)
        time.sleep(5)
        
        try:
            print("+" * 40)  # Shows in terminal when details of a new video is being scraped
            print("Scraping child links ")
            #scroll down to load comments
            driver.execute_script('window.scrollBy(0,1000);')
            time.sleep(15)
            #sort by top comments
            sort= driver.find_element(By.XPATH,"""//*[@id="icon-label"]""")
            sort.click()
            time.sleep(10)
            topcomments =driver.find_element(By.XPATH,"""//*[@id="menu"]/a[1]/paper-item/paper-item-body/div[1]""")
            topcomments.click()
            time.sleep(10)
            # Loads 20 comments , scroll two times to load next set of 40 comments. 
            for i in range(0,2):
                driver.execute_script("window.scrollTo(0,Math.max(document.documentElement.scrollHeight,document.body.scrollHeight,document.documentElement.clientHeight))")
                time.sleep(10)
            
            #count total number of comments and set index to number of comments if less than 50 otherwise set as 50. 
            totalcomments= len(driver.find_elements(By.XPATH,"""//*[@id="content-text"]"""))
         
            if totalcomments < 50:
                index= totalcomments
            else:
                index= 50 
                
            ccount = 0
            while ccount < index: 
                try:
                    comment = driver.find_elements(By.XPATH,'//*[@id="content-text"]')[ccount].text
                except:
                    comment = ""
                try:
                    authors = driver.find_elements(By.XPATH,'//a[@id="author-text"]/span')[ccount].text
                except:
                    authors = ""
                try:
                    comment_posted = driver.find_elements(By.XPATH,'//*[@id="published-time-text"]/a')[ccount].text
                except:
                    comment_posted = ""
                try:
                    replies = driver.find_elements(By.XPATH,'//*[@id="more-text"]')[ccount].text                    
                    if replies =="View reply":
                        replies= 1
                    else:
                        replies =replies.replace("View ","")
                        replies =replies.replace(" replies","")
                except:
                    replies = ""
                try:
                    upvotes = driver.find_elements(By.XPATH,'//*[@id="vote-count-middle"]')[ccount].text
                except:
                    upvotes = ""
                        
                youtube_dict['url'] = href[tcount]
                youtube_dict['link_title'] = title[tcount]
                youtube_dict['channel'] = channel[tcount]
                youtube_dict['no_of_views'] = numview[tcount]
                youtube_dict['time_uploaded'] =  postdate[tcount]
                youtube_dict['comment'] = comment
                youtube_dict['author'] = authors
                youtube_dict['comment_posted'] = comment_posted
                youtube_dict['no_of_replies'] = replies
                youtube_dict['upvotes'] = upvotes
                
                writer.writerow(youtube_dict.values())
                ccount = ccount +1
                
        except Exception as e:
            print(e)
            driver.close()
        tcount = tcount +1 
    print("Scrapping process Completed")   
    csv_file.close()    
except Exception as e:
    print(e)
    driver.close()

IndentationError: unexpected indent (Temp/ipykernel_39140/1701166377.py, line 2)

In [94]:
from selenium.webdriver.common.action_chains import ActionChains

button = driver.find_element(By.CLASS_NAME,"infoDismiss")
driver.implicitly_wait(10)
ActionChains(driver).move_to(button).click(button).perform()

InvalidSessionIdException: Message: invalid session id
Stacktrace:
Backtrace:
	Ordinal0 [0x005A78B3+2193587]
	Ordinal0 [0x00540681+1771137]
	Ordinal0 [0x00454070+802928]
	Ordinal0 [0x00477260+946784]
	Ordinal0 [0x0049D6A0+1103520]
	Ordinal0 [0x0049B4F5+1094901]
	Ordinal0 [0x0049B098+1093784]
	Ordinal0 [0x004355C7+677319]
	Ordinal0 [0x00435B43+678723]
	Ordinal0 [0x00435F7A+679802]
	GetHandleVerifier [0x00850CB2+2738370]
	GetHandleVerifier [0x008421B8+2678216]
	GetHandleVerifier [0x006317AA+512954]
	GetHandleVerifier [0x00630856+509030]
	Ordinal0 [0x0054743B+1799227]
	Ordinal0 [0x00435357+676695]
	Ordinal0 [0x00434E18+675352]
	GetHandleVerifier [0x008742BC+2883276]
	BaseThreadInitThunk [0x76F06739+25]
	RtlGetFullPathName_UEx [0x779B90AF+1215]
	RtlGetFullPathName_UEx [0x779B907D+1165]


In [ ]:
# open chrome 
youtube_pages = "https://www.youtube.com/"
locationOfWebdriver = (r"C:\Users\pranay\Desktop\chromedriver_win32 (1)\chromedriver.exe")
driver = webdriver.Chrome(locationOfWebdriver)
driver.get(youtube_pages)
time.sleep(10)
try:
    print("=" * 40)  # Shows in terminal when youtube summary page with search keyword is being scraped
    print("Scraping " + youtube_pages)
    search = driver.find_element_by_id('search')
    search.send_keys("Kishore Kumar")    
    driver.find_element_by_id('search-icon-legacy').click()
    time.sleep(20)    
    vtitle = driver.find_elements_by_id('video-title')
    subscription = driver.find_elements_by_id('byline')
    views = driver.find_elements_by_xpath('//div[@id="metadata-line"]/span[1]')
    posted = driver.find_elements_by_xpath('//div[@id="metadata-line"]/span[2]')
    
    tcount = 0
    href = []
    title = []
    channel = []
    numview = []
    postdate = []
        
    while tcount < 10:
        href.append(vtitle[tcount].get_attribute('href'))
        channel.append(subscription[tcount].get_attribute('title'))
        title.append(vtitle[tcount].text)
        numview.append(views[tcount].text)
        postdate.append(posted[tcount].text)  
        tcount = tcount +1
    
    # launch top ten extracted links and extract comment details
    tcount = 0    
    while tcount < 10:  
        youtube_dict ={}
        # extract comment section of top ten links
        url = href[tcount]
        print (url)
        driver.get(url)
        time.sleep(5)
        
        try:
            print("+" * 40)  # Shows in terminal when details of a new video is being scraped
            print("Scraping child links ")
            #scroll down to load comments
            driver.execute_script('window.scrollTo(0,390);')
            time.sleep(15)
            #sort by top comments
            sort= driver.find_element_by_xpath("""//*[@id="icon-label"]""")
            sort.click()
            time.sleep(10)
            topcomments =driver.find_element_by_xpath("""//*[@id="menu"]/a[1]/paper-item/paper-item-body/div[1]""")
            topcomments.click()
            time.sleep(10)
            # Loads 20 comments , scroll two times to load next set of 40 comments. 
            for i in range(0,2):
                driver.execute_script("window.scrollTo(0,Math.max(document.documentElement.scrollHeight,document.body.scrollHeight,document.documentElement.clientHeight))")
                time.sleep(10)
            
            #count total number of comments and set index to number of comments if less than 50 otherwise set as 50. 
            totalcomments= len(driver.find_elements_by_xpath("""//*[@id="content-text"]"""))
         
            if totalcomments < 50:
                index= totalcomments
            else:
                index= 50 
                
            ccount = 0
            while ccount < index: 
                try:
                    comment = driver.find_elements_by_xpath('//*[@id="content-text"]')[ccount].text
                except:
                    comment = ""
                try:
                    authors = driver.find_elements_by_xpath('//a[@id="author-text"]/span')[ccount].text
                except:
                    authors = ""
                try:
                    comment_posted = driver.find_elements_by_xpath('//*[@id="published-time-text"]/a')[ccount].text
                except:
                    comment_posted = ""
                try:
                    replies = driver.find_elements_by_xpath('//*[@id="more-text"]')[ccount].text                    
                    if replies =="View reply":
                        replies= 1
                    else:
                        replies =replies.replace("View ","")
                        replies =replies.replace(" replies","")
                except:
                    replies = ""
                try:
                    upvotes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')[ccount].text
                except:
                    upvotes = ""
                        
                youtube_dict['url'] = href[tcount]
                youtube_dict['link_title'] = title[tcount]
                youtube_dict['channel'] = channel[tcount]
                youtube_dict['no_of_views'] = numview[tcount]
                youtube_dict['time_uploaded'] =  postdate[tcount]
                youtube_dict['comment'] = comment
                youtube_dict['author'] = authors
                youtube_dict['comment_posted'] = comment_posted
                youtube_dict['no_of_replies'] = replies
                youtube_dict['upvotes'] = upvotes
                
                writer.writerow(youtube_dict.values())
                ccount = ccount +1
                
        except Exception as e:
            print(e)
            driver.close()
        tcount = tcount +1 
    print("Scrapping process Completed")   
    csv_file.close()    
except Exception as e:
    print(e)
    driver.close()

In [ ]:
for _ in range(20):
    driver.execute_script("window.scrollBy(0,1000)")
images=driver.find_elements(By.XPATH,'//img[@class="rg_iQ4LuWd"]')

img_urls=[]
img_data=[]
for image in images:
    source=image.get_attribute('src')
    if source is not None:
        if (source[0:4]=='http'):
            img_urls.append(source)
        
for i in range(len(img_urls)):
    if i >10:
        breakBy.XPATH,
    print('Downloading {0} of {1} images' .format(i,10))
    response=requests.get(img_urls[i])
    file=open(r"C:\Users\pranay\Desktop" + str(i)+".jpg","wb")
    file.write(response.content)
